# Run OpenMM on Google Colab

The purpose of this notebook is to show how to run an OpenMM simulation on Google Colab. This typically consists of the following steps:

1. Instal OpenMM.
2. Upload data files to the cloud.
3. Run the simulation.
4. Download the results.

In [ ]:
# Install OpenMM
import sys
print(sys.version)
!wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!bash Miniconda3-latest-Linux-x86_64.sh -bfp /usr/local
!conda config --set always_yes yes
!conda config --add channels omnia
!conda config --add channels conda-forge
!conda create -n openmm python=3.6 openmm
import sys
sys.path.append('/usr/local/envs/openmm/lib/python3.6/site-packages')
import simtk.testInstallation
simtk.testInstallation.main()

In [ ]:
# Import all the stuff we need
%matplotlib widget
from sys import stdout
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *

In [ ]:
# Upload the file alanine-dipeptide.pdb
from google.colab import files
uploaded = files.upload()

In [ ]:
# Copy the code from notebook2, example 3
pdb = PDBFile('alanine-dipeptide.pdb')
modeller = Modeller(pdb.topology, pdb.positions)
forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')
modeller.addSolvent(forcefield, model='tip3p', padding=1*nanometer)
print(modeller.topology)
# Write a PDB file to provide a topology of the solvated
# system to MDTraj below.
with open('init3.pdb', 'w') as outfile:
    PDBFile.writeFile(modeller.topology, modeller.positions, outfile)

# The modeller builds a periodic box with the solute and solvent molecules.
# PME is the method to compute long-range electristatic interactions in
# periodic systems.
system = forcefield.createSystem(modeller.topology, nonbondedMethod=PME, constraints=HBonds)
integrator = LangevinIntegrator(300*kelvin, 1/picosecond, 2*femtoseconds)
simulation = Simulation(modeller.topology, system, integrator)
simulation.context.setPositions(modeller.positions)
simulation.minimizeEnergy()
simulation.reporters.append(DCDReporter('traj3.dcd', 100))
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True,
        temperature=True, elapsedTime=True))
simulation.reporters.append(StateDataReporter("scalars3.csv", 100, time=True,
    potentialEnergy=True, totalEnergy=True, temperature=True))
simulation.step(100000)

In [ ]:
# Download the results.
# This is terrible slow for large files, such as traj3.dcd (29MB)
files.download('init3.pdb')
files.download('scalars3.csv')
files.download('traj3.dcd')